In [1]:
import pandas as pd
import numpy as np
import gc
import warnings
warnings.filterwarnings('ignore')

In [2]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [4]:
train = reduce_memory_usage(train)
test = reduce_memory_usage(test)

Mem. usage decreased to 505.45 Mb (76.9% reduction)
Mem. usage decreased to 252.25 Mb (76.9% reduction)


In [5]:
train.to_csv("train.csv", index = False)
test.to_csv("test.csv", index = False)

In [5]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from ngboost import NGBClassifier
from sklearn.tree import DecisionTreeClassifier
from ngboost.learners import default_tree_learner
from ngboost.distns import k_categorical
from ngboost.scores import MLE

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [6]:
X = train.iloc[:, 1:-1]
y = train.target

In [7]:
target = test.iloc[:, 1:]

In [8]:
del train
del test

In [9]:
X['std'] = X.std(axis=1)

target['std'] = target.std(axis=1)

In [10]:
skf = StratifiedKFold(n_splits = 10, random_state = 2021, shuffle = True)

In [11]:
cb = CatBoostClassifier(iterations = 10000, learning_rate = .03, random_state = 42)

In [12]:
cb_pred = np.zeros((target.shape[0]))
for i, idx in enumerate(zip(skf.split(X, y))) :
    tr_x, tr_y = X.iloc[idx[0][0]], y.iloc[idx[0][0]]
    val_x, val_y = X.iloc[idx[0][1]], y.iloc[idx[0][1]]
    
    cb.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], verbose = 1000, early_stopping_rounds = 500)
    
    val_pred = cb.predict_proba(val_x)[:, 1]
    
    score = roc_auc_score(val_y, val_pred)
    
    print(f'{i + 1} Fold roc_auc_score = {score}\n')
    
    fold_pred = cb.predict_proba(target)[:, 1]
    cb_pred += (fold_pred / 10)

0:	learn: 0.6808484	test: 0.6808337	test1: 0.6805923	best: 0.6805923 (0)	total: 211ms	remaining: 35m 7s
1000:	learn: 0.4677988	test: 0.4678248	test1: 0.4657574	best: 0.4657574 (1000)	total: 2m	remaining: 17m 59s
2000:	learn: 0.4591064	test: 0.4591326	test1: 0.4623964	best: 0.4623964 (2000)	total: 3m 54s	remaining: 15m 36s
3000:	learn: 0.4526649	test: 0.4526911	test1: 0.4615831	best: 0.4615831 (3000)	total: 5m 45s	remaining: 13m 26s
4000:	learn: 0.4468426	test: 0.4468691	test1: 0.4612089	best: 0.4611974 (3955)	total: 7m 36s	remaining: 11m 25s
5000:	learn: 0.4413650	test: 0.4413915	test1: 0.4610782	best: 0.4610678 (4908)	total: 9m 25s	remaining: 9m 24s
6000:	learn: 0.4360724	test: 0.4360988	test1: 0.4610146	best: 0.4609898 (5639)	total: 11m 14s	remaining: 7m 29s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.4609898391
bestIteration = 5639

Shrink model to first 5640 iterations.
1 Fold roc_auc_score = 0.8605092756331083
0:	learn: 0.6808018	test: 0.6808098	test1: 0.6

4000:	learn: 0.4462975	test: 0.4463240	test1: 0.4658964	best: 0.4658958 (3996)	total: 7m 38s	remaining: 11m 26s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.4658450465
bestIteration = 4413

Shrink model to first 4414 iterations.
10 Fold roc_auc_score = 0.8568038944304691


In [18]:
params = {
    'max_depth': 6,
    'subsample': 0.75,
    'colsample_bytree': 0.25,
    'colsample_bylevel': 0.6000000000000001,
    'min_child_weight': 56.41980735551558,
    'reg_lambda': 75.56651890088857,
    'reg_alpha': 0.11766857055687065,
    'gamma': 0.6407823221122686,
    'booster': 'gbtree',
    'eval_metric': 'auc',
    'use_label_encoder': False
    }

In [20]:
xgb = XGBClassifier(random_state = 42, n_estimators = 5000, learning_rate = .03, **params)

In [21]:
xgb_pred = np.zeros((target.shape[0]))
for i, idx in enumerate(zip(skf.split(X, y))) :
    tr_x, tr_y = X.iloc[idx[0][0]], y.iloc[idx[0][0]]
    val_x, val_y = X.iloc[idx[0][1]], y.iloc[idx[0][1]]
    
    xgb.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], verbose = 1000, early_stopping_rounds = 250)
    
    val_pred = xgb.predict_proba(np.float32(val_x))[:, 1]
    
    score = roc_auc_score(val_y, val_pred)
    
    print(f'{i + 1} Fold roc_auc_score = {score}\n')
    
    fold_pred = xgb.predict_proba(np.float32(target))[:, 1]
    xgb_pred += (fold_pred / 10)

[0]	validation_0-auc:0.78977	validation_1-auc:0.78745
[1000]	validation_0-auc:0.86430	validation_1-auc:0.85508
[2000]	validation_0-auc:0.87398	validation_1-auc:0.85634
[2231]	validation_0-auc:0.87560	validation_1-auc:0.85632
1 Fold roc_auc_score = 0.8563289464183201
[0]	validation_0-auc:0.78939	validation_1-auc:0.79071
[1000]	validation_0-auc:0.86416	validation_1-auc:0.85592
[2000]	validation_0-auc:0.87385	validation_1-auc:0.85720
[2647]	validation_0-auc:0.87818	validation_1-auc:0.85720
2 Fold roc_auc_score = 0.8572132578676128
[0]	validation_0-auc:0.78985	validation_1-auc:0.78685
[1000]	validation_0-auc:0.86436	validation_1-auc:0.85437
[2000]	validation_0-auc:0.87402	validation_1-auc:0.85566
[2748]	validation_0-auc:0.87900	validation_1-auc:0.85578
3 Fold roc_auc_score = 0.8557866762975239
[0]	validation_0-auc:0.78922	validation_1-auc:0.79030
[1000]	validation_0-auc:0.86408	validation_1-auc:0.85701
[2000]	validation_0-auc:0.87379	validation_1-auc:0.85800
[2610]	validation_0-auc:0.87794

In [11]:
params = {'n_estimators': 7095,
          'max_depth': 3,
          'learning_rate': 0.03474539414923071,
          'reg_alpha': 9.988737807614964,
          'reg_lambda': 8.888715867016865,
          'num_leaves': 109,
          'min_data_per_group': 133,
          'min_child_samples': 113,
          'colsample_bytree': 0.19592122573428444,
          'objective': 'binary',
          'metric': 'auc',}

In [12]:
lgbm = LGBMClassifier(random_state = 42, **params)

In [13]:
lgbm_pred = np.zeros((target.shape[0]))
for i, idx in enumerate(zip(skf.split(X, y))) :
    tr_x, tr_y = X.iloc[idx[0][0]], y.iloc[idx[0][0]]
    val_x, val_y = X.iloc[idx[0][1]], y.iloc[idx[0][1]]
    
    lgbm.fit(tr_x, tr_y, eval_set = [(tr_x, tr_y), (val_x, val_y)], verbose = 1000, early_stopping_rounds = 500)
    
    val_pred = lgbm.predict_proba(val_x)[:, 1]
    
    score = roc_auc_score(val_y, val_pred)
    
    print(f'{i + 1} Fold roc_auc_score = {score}\n')
    
    fold_pred = lgbm.predict_proba(target)[:, 1]
    lgbm_pred += (fold_pred / 10)

Training until validation scores don't improve for 500 rounds
[1000]	training's auc: 0.855152	valid_1's auc: 0.851902
[2000]	training's auc: 0.859754	valid_1's auc: 0.854986
[3000]	training's auc: 0.862113	valid_1's auc: 0.855906
[4000]	training's auc: 0.863785	valid_1's auc: 0.85622
[5000]	training's auc: 0.86518	valid_1's auc: 0.856286
Early stopping, best iteration is:
[5015]	training's auc: 0.865201	valid_1's auc: 0.85629
1 Fold roc_auc_score = 0.8562900727809859

Training until validation scores don't improve for 500 rounds
[1000]	training's auc: 0.855038	valid_1's auc: 0.85302
[2000]	training's auc: 0.859671	valid_1's auc: 0.855812
[3000]	training's auc: 0.86201	valid_1's auc: 0.856662
[4000]	training's auc: 0.863676	valid_1's auc: 0.85697
[5000]	training's auc: 0.865078	valid_1's auc: 0.857072
[6000]	training's auc: 0.866344	valid_1's auc: 0.857068
Early stopping, best iteration is:
[5705]	training's auc: 0.865975	valid_1's auc: 0.857081
2 Fold roc_auc_score = 0.8570806239402313

In [14]:
submission = pd.read_csv('sample_submission.csv')

In [15]:
ens1 = pd.read_csv("1014_1st.csv")

In [18]:
submission['target'] = ens1['target'] * (2/3) + lgbm_pred * (1/3)

In [21]:
submission.to_csv("1018_2nd.csv", index = False)